# Legal assistant explorer

Copyright 2025, Denis Rothman

This notebook serves as an interactive explorer for the **Legal Compliance Assistant**, an advanced multi-agent system equipped with essential production safeguards like a **two-stage content moderation protocol**

You will use a series of "Control Decks" to test the engine's core capabilities in a legal context, including high-fidelity RAG, context reduction, and grounded reasoning.

Crucially, you will also explore the system's realistic limitations through "limit tests" designed to simulate the real-world "headaches" that arise when AI confronts complex and ambiguous organizational challenges. This hands-on experience demonstrates how to apply, validate, and understand the boundaries of a production-ready Context Engine.

*Note* Make sure to run `Data_Ingestion.ipynb` first to ingest the data for this notebook.

# I. Inititalization

## GitHub

In [1]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1823  100  1823    0     0   7900      0 --:--:-- --:--:-- --:--:--  7891
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  13488      0 --:--:-- --:--:-- --:--:-- 13497
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9440  100  9440    0     0  22360      0 --:--:-- --:--:-- --:--:-- 22316
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3611  100  3611    0     0  14924      0 --:--:-- --:--:-- --:--:-- 14983
  % Total    % Received % Xferd  Average Speed   Tim

## Installation and client setup

In [2]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [3]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [4]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
import json  # <--- Added for type handling
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine, now with robust type handling for moderation.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        # Goal is always a string, so this is safe
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")

        # HANDLE NON-STRING OUTPUTS ---
        # If the agent returns a dictionary (JSON), convert it to a string first.
        text_to_check = result
        if isinstance(result, (dict, list)):
            try:
                text_to_check = json.dumps(result, indent=2)
            except TypeError:
                text_to_check = str(result)
        # ------------------------------------------

        moderation_report = helpers.helper_moderate_content(text_to_moderate=text_to_check, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        # If result is still a dict (and wasn't redacted), display it nicely
        if isinstance(result, dict):
             display(Markdown(f"```json\n{json.dumps(result, indent=2)}\n```"))
        else:
             display(Markdown(str(result)))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)

## Control Deck configuration

In [5]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [6]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.

# November 20, 2025 goal update:
# The former goal contained the the term "summarize" creating ambiguity.
#goal = "Summarize the key points of the Non-Disclosure Agreement."

# The new goal now explains that the task is not to directly summarize but first retrieve data thus clarifying the ambiguity
goal = "First, retrieve the content of the Non-Disclosure Agreement (NDA) from the knowledge base. Then, summarize its key points."
# The system will then function correctly, sanitize the information and continue the process

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4


--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment

```json
{
  "summary": "Here are the key points, based solely on the confidentiality clause found in the Service Agreement:\n\n1. **Purpose**  \n   - To keep proprietary information shared during the Service Agreement confidential and prevent unauthorized disclosure to third parties.\n\n2. **Definition of Confidential Information**  \n   - Described generally as \u201call proprietary information disclosed during the term of this Agreement.\u201d  \n   - No further detail or specific categories are provided.\n\n3. **Obligations of the Receiving Party**  \n   - Both parties must:\n     - Maintain the confidentiality of proprietary information.\n     - Not disclose such information to any third party without prior written consent of the disclosing party.\n\n4. **Exclusions from Confidentiality**  \n   - No explicit exclusions (e.g., publicly known information, independently developed information) are stated in the available text.\n\n5. **Permitted Disclosures**  \n   - Disclosure to third parties is only permitted with prior written consent from the disclosing party.\n\n6. **Term and Duration of Obligations**  \n   - The clause applies to information disclosed \u201cduring the term of this Agreement.\u201d  \n   - No explicit statement about whether confidentiality obligations survive termination or for how long.\n\n7. **Return or Destruction of Information**  \n   - No provision in the available text regarding return or destruction of confidential information.\n\n8. **Remedies for Breach**  \n   - No specific remedies (e.g., injunctive relief, damages, indemnification) are stated in the available text.\n\n9. **Governing Law / Jurisdiction**  \n   - No governing law or jurisdiction clause appears in the confidentiality language provided."
}
```



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 14.63 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'There is no standalone Non-Disclosure '
                                       'Agreement (NDA) text in the provided '
                                       'sources. The only relevant '
                                       'confidentiality language appears as a '
                                       '**confidentiality clause within the '
                                       'Service Agreement**, not as a separate '
                                       'NDA document.\n'
                                       '\n'
                                       'The full confidentiality-related '
                                       'content from the sources is:\n'
                                       '\n'
                                       '> **From the Service Agreement**  \n'
                   

In [7]:
#@title CONTROL DECK TEMPLATE 1: High-Fidelity RAG

# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

# === CONTROL DECK 1: High-Fidelity RAG in a Legal Context ===
goal = "What are the key confidentiality obligations in the Service Agreement v1, and what is the termination notice period? Please cite your sources."

# === CONTROL DECK 1 (LIMIT TEST): Sanitization of Legal Testimony ===
#goal = "What did Mr. Smith advise his client regarding the assets?"

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Here’s the quick rundown:

**Confidentiality (Clause 4)**  
- Both sides have to keep each other’s proprietary info secret during the term of the agreement.  
- You can’t share that info with any third party unless you get prior written consent.  
- It basically covers all proprietary info shared during the agreement, but it doesn’t spell out what counts as “proprietary” or list any exclusions.  
- There’s no detail on how carefully you have to protect it (no standard of care), and nothing about returning or destroying info at the end.  
- It also doesn’t say how long the duty of confidentiality lasts after the agreement ends.  
- The only real exception is if the other party says “yes” in writing—no carve-outs for public info, prior knowledge, legal requirements, etc.

**Termination (Clause 5)**  
- Either party can terminate the agreement for any reason with 30 days’ written notice.  
- There’s no separate “for cause” or “material breach” clause and no cure period—just the same 30‑day notice for everything.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 53.35 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Below are the requested extractions '
                                       'and notes from the document that '
                                       'matches “Service Agreement v1” based '
                                       'on the provided text.\n'
                                       '\n'
                                       '---\n'
                                       '\n'
                                       '## (a) Confidentiality / '
                                       'Non‑Disclosure\n'
                                       '\n'
                                       '### Relevant Clause and Exact Text\n'
                                       '\n'
                                       '- **Clause number:** 4  \n'
                                       '- **Exact clause text (ve

In [8]:
#@title CONTROL DECK TEMPLATE 2: Context Reduction

# 1. Define the Goal: A multi-step task that involves summarizing a large
#    document and then using that summary for a different purpose.
#    - DOMAIN: Any field with large documents (legal, scientific, corporate).
#    - KEY CAPABILITY: Tests the `Summarizer` agent and the engine's ability
#      to perform Context Chaining between the `Summarizer` and the `Writer`.
# goal = "[INSERT YOUR CONTEXT REDUCTION GOAL HERE]"

# === CONTROL DECK 2: Context Reduction for Client Communication ===
goal = "First, summarize the Provider Inc. Privacy Policy. Then, using ONLY the information in that summary, draft a short, client-facing paragraph for a website FAQ that explains our data retention policy in simple, non-legalistic terms."

# === CONTROL DECK 2 (LIMIT TEST): The Vague Objective ===
#goal = "Summarize the service agreement and then write a story about it."


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Here’s the quick rundown:

- They keep two kinds of data: stuff you give them (like your name and email) and stuff that’s collected automatically (like your IP address and browsing history).  
- They hold onto it as long as they need it to run and improve the service and to meet legal, accounting, and reporting rules.  
- Generally, they won’t keep your data more than 5 years after your last interaction with their service.  
- They say they only keep what’s necessary, but they don’t really explain *how* or *when* things are actually deleted or anonymized, and they don’t mention any clear process for you to request deletion.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 19.87 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Here is the full text of the Provider '
                                       'Inc. Privacy Policy based on the '
                                       'provided source, with emphasis on data '
                                       'retention, storage duration, deletion, '
                                       'and archival practices where '
                                       'specified.\n'
                                       '\n'
                                       '---\n'
                                       '\n'
                                       '**Privacy Policy for Provider '
                                       'Inc.**  \n'
                                       '**Last Updated: May 15, 2025**\n'
                                       '\n'
                                     

In [9]:
#@title CONTROL DECK TEMPLATE 3: Grounded Reasoning & Hallucination Prevention

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# === CONTROL DECK 3: Grounded Reasoning and Hallucination Prevention ===
goal = "Write a persuasive opening statement for a trial involving a monkey that can fly a rocket."

# === CONTROL DECK 3 (LIMIT TEST): The Ambiguous Request ===
#goal = "Analyze the attached NDA and draft a pleading based on its terms."

# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


The room was too quiet.

Twelve chairs. Twelve faces. All watching.

I stood. My knees felt loose, like they might give at any second. The air tasted like dust and something metallic. The clock on the wall ticked too loud. Each second snapped.

“Ladies and gentlemen of the jury…”

My own voice sounded strange. Not quite mine. Too big in the still air, bouncing off the wood and the white walls.

I told them my name. Who I stood for. Just words, but they felt heavy, like stacking bricks. Behind me, I could feel my client’s presence without looking. A slow breath. A tiny shift of weight. Fear, held very still.

“This case,” I said, “is about a promise that was made… and broken.”

The words hung there. A simple line, but the whole room seemed to bend around it. Broken promises. Everyone knows that feeling. Everyone has one they remember.

I could sense the other side across the room. Quiet. Waiting. A different story, ready to be told. Their story. Their version. Their shadows.

I laid out what would come, keeping my voice steady. First the story. Then the rules. Then what I would ask them to do. A path through a dark place. I tried to sound like a guide. Not a fighter.

My eyes moved from face to face. A woman in a blue sweater, hands clasped tight. A man with a gray beard, staring at me without blinking. A younger man tapping one foot, heel ticking against the floor in a dull, nervous beat.

They didn’t know the story yet. But I did. I carried it like a weight in my chest.

“It starts simple,” I said. “A person. A job. A promise.”

I walked them through it, piece by piece. No big words. Just people and choices and time.

“This happened on a Tuesday,” I said. “Early. Still dark outside. The office lights were the only ones on in the building. You’ll hear how the hallway was quiet, how the hum of the fluorescent lights was the loudest sound.”

I could almost hear it myself. That lonely electric buzz. The faint, hollow echo of footsteps.

“You’ll hear about my client,” I went on. I said their name. Their work. Their family. The late nights. The early mornings. The way they trusted the people around them. Trusted the system that was supposed to protect them.

As I spoke, I gave the jury small details. Coffee gone cold in a chipped mug. A clock glowing 4:17 a.m. on a dim screen. The smell of paper and toner. A phone that rang once… then stopped.

Simple things. Human things. The texture of an ordinary life. The kind of life that does not expect danger.

But danger was there. Waiting. Quiet.

I could feel it shaping the story even as I tried to keep my voice calm. The threat was not a monster in the dark. It was something colder. A choice someone made. A rule someone broke. A promise someone decided did not matter.

“You’ll see who did what,” I said. “You’ll see when they chose to look away. When they chose not to fix what they knew was wrong.”

The room seemed to lean in, just a fraction.

I did not shout. I did not point. I only drew the outline of what they would hear.

“You will hear from witnesses,” I said. “From people who were there in those halls. People who saw how the rules were ignored. How warnings were brushed aside.”

I mentioned the documents. Emails sent at odd hours. Messages marked “urgent” that went nowhere. Dates. Times. Little black letters on glowing screens.

“You will see those words,” I told them. “You will see what they knew. When they knew it. And what they chose to do anyway.”

The threat was not loud. It was patient. It lived in silence, in the spaces where someone should have acted and did not. It crept forward in every ignored message, every quiet shrug, every “we’ll deal with that later.”

I knew the other side would talk about other things. They would try to twist the shadows. Call them something else. Say the danger wasn’t real. That my client knew the risk. That it wasn’t so bad.

I stepped into that darkness before they could.

“You will also hear some hard things about my client,” I said. “You’ll hear that they didn’t complain right away. That they tried to keep going. That they tried to trust that someone would do the right thing.”

I let that hang. No excuse. Just a truth.

“You’ll hear that they stayed. Because this was their job. Their life. Their only way to provide for their family.”

I put those facts where they belonged. In our story. Not theirs.

Little noises in the courtroom grew louder in the pause. A chair creaking. Paper sliding. Someone clearing their throat. The steady scratch of the court reporter’s keys, like rain on glass.

Then I brought in the rules. Not as laws from a book, but as simple lines.

“The judge will tell you,” I said, “that to hold someone responsible here, you will have to decide three things.”

I lifted one finger. Then another. Then another. Simple. Measured.

“First, did they have a duty to keep people safe? Second, did they break that duty? Third, did that choice hurt my client?”

Three steps. Three questions.

“You will see that duty in their own words,” I said. “In their policies. In their training. In what they promised to do.”

I connected each rule to a moment in the story.

“You will see they broke that duty when they ignored clear warnings. When they let a known danger stay in the dark. When they chose to save time and money instead of fixing what they knew was wrong.”

“And you will see what that did to my client. Not only to their body, but to their life. To their sleep. To their sense of safety when they walk down a hallway that looks just like the one where this all began.”

The threat was not over. That was the chilling part. The danger my client met still waited there, in that place, for the next person who trusted the same broken promise.

That quiet idea slid through the room like a draft of cold air.

“This case is not only about the past,” I said softly. “It is about what happens if nothing changes.”

I talked about burdens, but in everyday words.

“In this courtroom, we have to prove what happened,” I said. “You don’t have to prove anything. You are the ones who listen. Who watch. Who weigh.”

I didn’t ask them to take my word for it. I told them they would see. They would hear. They would feel the story settle inside them as the days went on, each piece clicking into place.

At the end, I stepped closer to what I needed from them. The ask. The verdict.

“When all the evidence is in,” I said, “we will come back to you and ask for one thing.”

I paused. The fluorescent lights hummed. The air conditioner kicked on with a low, distant growl. Somewhere in the back, a pen dropped, then rolled, then stopped.

“We will ask you to say, with your verdict, that promises like this matter,” I said. “That when someone is trusted with safety, and they break that trust, they must be held accountable.”

I did not talk about anger. I talked about fairness. About the community beyond the walls.

“Your decision will not just speak for my client,” I said. “It will speak for everyone who walks into a place believing the people in charge have done what they said they would do.”

The threat was there, just under the surface: if they did nothing, silence would win again. The same shadows would spread. The same quiet harm would go on, unseen, until it was too late for someone else.

I pulled the story closed, tight and simple.

“This case,” I repeated, “is about a promise that was made. A promise that was broken. And the cost of that choice.”

Then I stopped.

For a moment, no one moved. The clock kept ticking. The lights kept buzzing. The shadows along the ceiling stayed where they were, long and still.

The danger, the wrong, the quiet breaking of trust—it all waited there in the silence between us, in the few feet of space between the jury box and where I stood.

Soon, they would hear every detail. Every sound. Every word. But for now, they had only the outline. The shape of what was coming.

And they knew, as I did, that once the story started, they would not be able to look away.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 102.99 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Increase tension and create '
                                  'suspense.", "style_guide": "Use short, '
                                  'sharp sentences. Focus on sensory details '
                                  '(sounds, shadows). Maintain a slightly '
                                  'eerie but age-appropriate tone.", '
                                  '"participants": [{"role": "Agent", '
                                  '"description": "The protagonist '
                                  'experiencing the events."}, {"role": '
                                  '"Source_of_Threat", "description": "The '
                                  'underlying danger or mystery."}], '
                                  '"instruction": "Rewrite the provided facts '
                              